In [8]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

#試取資料
file = open("/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/pingpong/log//1p_platedontmove1.pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml_1P'])

dict

In [9]:
game_info = data["ml_1P"]["scene_info"]
game_command = data["ml_1P"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (60, 240)}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (65, 240)}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (70, 240)}, {'frame': 3, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (75, 240)}, {'frame': 4, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (80, 240)}, {'frame': 5, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (85, 240)}, {'frame': 6, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform

In [10]:
for i in range(2, 100):
    path = "/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/pingpong/log/1p_platedontmove" + str(i) + ".pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_1P']['scene_info']
    game_command = game_command + data['ml_1P']['command']
    file.close()
    
print(len(game_info))
print(len(game_command))

126320
126320


In [11]:
g = game_info[1]

feature = np.array([g["ball"][0], g["ball"][1], g["ball_speed"][0], g["ball_speed"][1], g["platform_1P"][0]+15,80])
print(feature)
print(game_command[1])
if game_command[1] == "NONE":
	game_command[1] = 0
elif game_command[1] == "MOVE_LEFT":
	game_command[1] = 1
else:
	game_command[1] = 2

[ 98 415   0   0  95  80]
None


In [12]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    ball_x=g["ball"][0]
    ball_y=g["ball"][1]
    ball_vx=g['ball'][0] - g_last['ball'][0]
    ball_vy=g['ball'][1] - g_last['ball'][1]
    des_x = 80
    
    if ball_vy > 0:                                             #ball go down
            if ball_vx < 0:                                         #ball move left
                des_x = ball_x - (420 - ball_y)
            else:                                                       #ball move right
                des_x = ball_x + (420 - ball_y)
    else:                                                           #ball move up  
            if ball_vx < 0:
                des_x = ball_x - (680 - (420 - ball_y))
            else:
                des_x = ball_x + (680 - (420 - ball_y))
    while des_x > 200 or des_x < 0:
        if des_x > 200:
                des_x = 200 - (des_x - 200)
        else: 
                des_x = -des_x
    feature = np.vstack((feature, [g['ball'][0], g['ball'][1],ball_vx, ball_vy,g['platform_1P'][0]+15,des_x]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "MOVE_LEFT": game_command[i] = 1
    else: game_command[i] = 2 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 98 415   0   0  95  80]
 [ 98 415   0   0  95  27]
 [ 98 415   0   0  95  27]
 ...
 [ 66 107  22 -22 120  33]
 [ 88  85  22 -22 115  33]
 [110  63  22 -22 110  33]]
(126318, 6)
[2 2 2 ... 1 1 1]
(126318,)


In [13]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9) #9
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=6) #n_jobs為平行運算的數量 6
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model_knn1.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    2.8s remaining:    5.5s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    2.8s remaining:    2.8s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    2.8s remaining:    1.4s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    2.9s finished


In [14]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 1}
[[18634    34    41]
 [   56  7464     0]
 [   62     1 11604]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18709
           1       1.00      0.99      0.99      7520
           2       1.00      0.99      1.00     11667

    accuracy                           0.99     37896
   macro avg       1.00      0.99      0.99     37896
weighted avg       0.99      0.99      0.99     37896

